## 5-2 교차 검증과 그리드 서치

#### 검증 세트(validation set) : 테스트 세트를 사용하지 않고 과대/과소 적합을 판단하기 위해 훈련 세트를 다시 나눈 데이터 (훈련세트 중 20%를 떼어놈)

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
sub_input.shape, val_input.shape

((4157, 3), (1040, 3))

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 훈련세트에 과대적합되어 있다.

0.9971133028626413
0.864423076923077


#### 교차 검증(cross validation) : 검증세트를 떼어 내어 평가하는 과정을 반복하고 이 점수를 평균하여 최종 검증 점수 획득

- cross_validate(model, train data set) (-> 훈련 세트를 섞어서 폴드를 나누지 않기때문에 분할기를 지정해야함)
- 회귀 모델의 경우, KFold 분할기 사용. 분류 모델일 경우, StratifiedKFold 사용

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

# fit_time : 모델을 훈련하는 시간
# score_time : 모델을 검증하는 시간
# test_score : 검증 폴드의 점수

{'fit_time': array([0.00447106, 0.00459194, 0.00530291, 0.00477695, 0.0043211 ]),
 'score_time': array([0.00033689, 0.00045729, 0.00051999, 0.00031805, 0.00029993]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [8]:
# 교차 검증의 최종 점수는 test_score 키에 담긴 점수들을 평균하여 얻을 수 있다.

import numpy as np
np.mean(scores['test_score'])

0.855300214703487

In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [10]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행해보자

splitter = StratifiedKFold(n_splits=10, shuffle = True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

#### 하이퍼 파라미터 튜닝
- 하이퍼 파라미터 : ML 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터

- GridSearchCV 클래스 : 하이퍼파라미터 탐색과 교차 검증을 함께 수행. 매개변수가 많아질 경우 매개변수들을 동시에 바꿔가며 최적 값을 찾는 것이 복잡해지는 것을 해결해준다

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# GridSearchCV의 cv 매개변수 default는 5. 따라서 위 코드에서는 min_impurity_decrease 값 마다 5-폴드 교차검증을 수행하므로 5x5=25개의 모델을 훈련한다.
# n_jobs = 병렬 실행에 사용할 CPU 코어수 지정. (default : 1, -1로 지정시 시스템 내 모든 코어 사용)

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_
# GridSearch는 훈련이 끝나면 훈련한 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 다시 모델을 훈련한다.
# 이 모델은 gs객체의 best_estimator_속성에 저장되어있다.
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
gs.best_params_ # GridSearch가 찾은 최적의 파라미터

{'min_impurity_decrease': 0.0001}

In [16]:
gs.cv_results_['mean_test_score'] # 각 매개변수에서 수행한 교차검증의 평균점수

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # np.argmax(): 가장 큰 값의 인덱스 추출
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

##### 정리
- 1. 먼저 탐색할 매개변수를 지정한다.
- 2. 훈련 세트에서 grid search를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. (이 조합은 grid search 객체에 저장됨)
- 3. grid search는 교차 검증에 사용한 훈련세트가 아닌 전체 훈련 세트를 사용해 최종 모델을 훈련한다. (이 모델은 grid search 객체에 저장됨)

In [18]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), # 9개
         'max_depth' : range(5, 20, 1), # 15개
         'min_samples_split' : range(2, 100, 10)} # 10개
## 이 매개변수로 수행할 교차 검증 횟수 : 9 x 15 x 10 = 1350
## GridSearchCV의 cv 매개변수 default는 5 -> 만들어지는 모델의 수 1350 x 5 = 6750

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [21]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

- 랜덤 서치(random search) : 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어렵고 매개변수 조건이 너무 많아서 gridsearch 수행 시간이 오래 걸릴 때 사용하면 좋다. 매개변수 값의 목록이 아닌 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

##### scipy library : 파이썬의 핵심 과학 라이브러리 중 하나. 적분 , 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리
- uniform : 주어진 범위에서 고르게 실수값을 뽑는다. -> 균등분포에서 샘플링한다.
- randint : 주어진 범위에서 고르게 정수값을 뽑는다. -> 균등분포에서 샘플링한다.

In [22]:
from scipy.stats import uniform, randint

In [23]:
rgen = randint(0,10)
rgen.rvs(10)

array([6, 9, 8, 3, 0, 7, 1, 0, 9, 7])

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106,  96,  96, 110,  99, 100, 114,  92,  92,  95]))

In [25]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.31851972, 0.10172389, 0.88326878, 0.98184474, 0.23547778,
       0.30436593, 0.04707984, 0.57720671, 0.30329533, 0.7709483 ])

In [26]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
         'max_depth' : randint(20, 50),
         'min_samples_split' : randint(2, 25),
         'min_samples_leaf' : randint(1, 25)
         }

In [27]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs = -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8050448370>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f806b45a970>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8050448340>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f80582633d0>},
                   random_state=42)

In [28]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [29]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [30]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.86

- DecisionTreeClassifier 클래스에 splitter='random'매개변수를 추가하고 다시 훈련해보자

In [31]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter = 'random'), params, n_iter=100, n_jobs = -1,random_state=42)

In [32]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8050448370>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f806b45a970>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8050448340>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f80582633d0>},
                   random_state=42)

In [33]:
dt = gs.best_estimator_
dt.score(test_input, test_target)
# 테스트 세트에서 성능이 떨어졌다 !

0.786923076923077

## 5-3 트리의 앙상블

- 정형 데이터(structured data) : 구조로 되어 있는 데이터. csv나 database, excel 등에 저장하기 쉬움
- 비정형 데이터(unstructured data) : database나 excel로 표현하기 어려움. ex) 텍스트 데이터 사진, 디지털 음악....

- 앙상블 학습(ensemble learning) : 정형 데이터를 다루는데 적합한 알고리즘. 대부분 결정 트리를 기반으로 만들어져 있음

- 랜덤 포레스트(random forest) : 앙상블 학습의 대표 알고리즘. 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든 후 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

- 부트스트랩(bootstrap) : 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine-date')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [35]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

# 훈련세트에 과대적합되었다.

(0.9973541965122431, 0.8905151032797809)

In [36]:
rf.fit(train_input, train_target)
rf.feature_importances_ # 특성 중요도

array([0.23167441, 0.50039841, 0.26792718])

In [37]:
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
rf.oob_score_

# 부트스트랩샘플에 포함되지 않고 남는 샘플 : OOB 샘플(Out Of Bag) 
# OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정트리를 평가할 수 있다 !!

0.8934000384837406

#### 엑스트라 트리(Extra Tree)
- 기본적으로 100개의 결정 트리를 훈련한다. 
- 랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수를 지원한다. 
- 또한 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용한다. 
- 랜덤 포레스트와 달리 부트스트랩 샘플을 사용하지 않는다. 즉, 각 결정 트리를 만들 때 전체 훈련세트를 사용한다
- 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

In [38]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9974503966084433, 0.8887848893166506)

In [40]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

### 그레이디언트 부스팅(Gradient Boosting)
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 방법
- 기본적으로 깊이가 3인 결정 트리 100개 사용한다.
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

In [41]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.8881086892152563, 0.8720430147331015)

In [42]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.11949946, 0.74871836, 0.13178218])

#### 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)
- 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘
- 입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
- 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용한다. 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.
- 트리의 개수를 지정하는데 n_estimators 대신에 부스팅 반복횟수를 지정하는 max_iter를 사용한다.

In [43]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9321723946453317, 0.8801241948619236)

permutation_importance()
- 히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산해보자
- 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지를 계산한다.
- 훈련 세트 뿐만아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있다.
- n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정한다.

In [46]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean

# importances_: 특성 중요도
# importances_mean : 평균
# importances_std : 표준편차

array([0.08876275, 0.23438522, 0.08027708])

In [48]:
# 이번엔 테스트 세트에서의 특성 중요도를 계산해보자

result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean

array([0.05969231, 0.20238462, 0.049     ])

In [47]:
hgb.score(test_input, test_target)

0.8723076923076923

In [8]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score= True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

ModuleNotFoundError: No module named 'xgboost'

In [5]:
!pip3 install xgboost

distutils: /Library/Frameworks/Python.framework/Versions/3.9/include/python3.9/UNKNOWN
sysconfig: /Library/Frameworks/Python.framework/Versions/3.9/include/python3.9
user = False
home = None
root = None
prefix = None
distutils: /Library/Frameworks/Python.framework/Versions/3.9/include/python3.9/UNKNOWN
sysconfig: /Library/Frameworks/Python.framework/Versions/3.9/include/python3.9
user = False
home = None
root = None
prefix = None
